---
date: 2024-04-15
title: DWARF Verification via Ghidra and CBMC
---

[DWARF](https://dwarfstd.org/) is debug data. It is [under appreciated](https://www.philipzucker.com/dwarf-patching/). Debug data is a translation validation artifact. By design, it leaves breadcrumbs of how the low (assembly) code connects to the high (C) code.

Much of the information is currently nonrigorous, at a best effort by compilers writers for human debugger consumption. It _could_ be better though.

"Where Did My Variable Go? Poking Holes in Incomplete Debug Information" https://arxiv.org/pdf/2211.09568.pdf is a really interesting paper on techniques to validate dwarf debug information. See the references and citations for more

Checking debug info dynamically is conceptually pretty straightforward. Devil in details though.
One approach is to compare `-O0` execution/debug data with higher optimization levels using. Presumably `-O0` is the less likely to omit or incorrectly generate debug data.
You could also run an assembly program in a debugger in sync with a C interpreter (like https://github.com/kframework/c-semantics  https://www.cl.cam.ac.uk/~pes20/cerberus/ https://www.reddit.com/r/ProgrammingLanguages/comments/hf441y/an_interpreter_for_c/ ). Set a breakpoint at every position the debug data should have an opinion and compare. Maybe throw a fuzzer on it.

These are all viable, but because of the particular hammer I've been forging, it makes sense to do this same process using CBMC, the C bounded model checker.

I've been working on a tool [pcode2c](https://github.com/philzook58/pcode2c) which uses [ghidra pcode](https://github.com/angr/pypcode) lifting and generates a specialized C interpreter for that binary. In other words, static binary translation to C. This is not "decompilation" really, because I make zero effort to make readable idiomatic C, recover loops, recover types. All of that decreases the connection to the original binary.

Here is a baby C example from  https://www.mimuw.edu.pl/~alx/konstruowanie/ACSL-by-Example.pdf It's a function that takes in a size parameter, and checks if two arrays of that size are equal.

In [4]:
%%file /tmp/all_equal.c
#include <stdbool.h>
#include <assert.h>
bool equal(int n, int a[], int b[]){
    for(int i = 0; i < n; i++){
        if(a[i] != b[i]){
            return false;
        }
    }
    return true;
}

int main(){
    int a[10]; // = {42}; //calloc(10*sizeof(int));
    assert(equal(10, a, a));
}


Writing /tmp/all_equal.c


In [5]:
!gcc -Og -g -Wall -Wextra -std=c11 /tmp/all_equal.c -o /tmp/all_equal

We can take a peek at the assembly

In [25]:
!objdump -l -F -d /tmp/all_equal | grep -A 24 '001169 <equal>'

0000000000001169 <equal> (File Offset: 0x1169):
equal():
/tmp/all_equal.c:3
    1169:	f3 0f 1e fa          	endbr64 
/tmp/all_equal.c:4
    116d:	b8 00 00 00 00       	mov    $0x0,%eax
    1172:	eb 03                	jmp    1177 <equal+0xe> (File Offset: 0x1177)
/tmp/all_equal.c:4 (discriminator 2)
    1174:	83 c0 01             	add    $0x1,%eax
/tmp/all_equal.c:4 (discriminator 1)
    1177:	39 f8                	cmp    %edi,%eax
    1179:	7d 13                	jge    118e <equal+0x25> (File Offset: 0x118e)
/tmp/all_equal.c:5
    117b:	48 63 c8             	movslq %eax,%rcx
    117e:	44 8b 04 8a          	mov    (%rdx,%rcx,4),%r8d
    1182:	44 39 04 8e          	cmp    %r8d,(%rsi,%rcx,4)
    1186:	74 ec                	je     1174 <equal+0xb> (File Offset: 0x1174)
/tmp/all_equal.c:6
    1188:	b8 00 00 00 00       	mov    $0x0,%eax
/tmp/all_equal.c:10
    118d:	c3                   	ret    
/tmp/all_equal.c:9
    118e:	b8 01 00 00 00       	mov    $0x1,%eax
    1193:	c3                

A quick summary of the assembly:
- The first arguments are in rdi,rsi,rdx. These are `n` `a` `b` respectively.
- First the loop iniitalizes by setting `i` in `eax` to 0. Then it jumps to the loop break check
- 0x1177 compares `i` with `n`
- The body of the loop is in 0x117b. It loads memory from `a[i] into `r8d` and then compares it with with memory in `b[i]`
- A jump returns to loop top at 0x1174 where `i` in `%eax` is incremented if the are equal
- Otherwise it returns false=0 in `eax` at `0x118d`
- If the loop is done it jumps from 0x1179 to 0x118e and returns 1 in `%eax`

I can dump this assembly out into a C interpreter using my pcode2c tool. You can see there is a very direct correspondence to the assembly (many C lines to 1 assembly line). There is a central switch-case that siwtches on the program counter and performs the pocde operations. 

In [9]:
!python3 -m pcode2c /tmp/all_equal --start-address 0x1169 --file-offset 0x1169 --size 0x2b > /tmp/all_equal_pcode.c

In [10]:
!cat /tmp/all_equal_pcode.c

/* AUTOGENERATED. DO NOT EDIT. */
#include "_pcode.h"
void pcode2c(CPUState *state, size_t breakpoint){
    uint8_t* register_space = state->reg; 
    uint8_t* unique_space = state->unique; 
    uint8_t* ram_space = state->ram;
    uint8_t* ram = ram_space; // TODO: remove this
    for(;;){
    switch(state->pc){
        case 0x1169:
        L_0x1169ul:
            INSN(0x1169ul,"0x1169: ENDBR64 ")
        case 0x116d:
        L_0x116dul:
            INSN(0x116dul,"0x116d: MOV EAX,0x0")
            L_P_0x1: COPY(register_space + 0x0ul /* RAX */, 8ul, (uint8_t *)&(long unsigned int){0x0ul}, 0x8ul);
        case 0x1172:
        L_0x1172ul:
            INSN(0x1172ul,"0x1172: JMP 0x1177")
            L_P_0x2: BRANCH_GOTO_TEMPFIX(ram_space, 0x1177ul, 8ul);
        case 0x1174:
        L_0x1174ul:
            INSN(0x1174ul,"0x1174: ADD EAX,0x1")
            L_P_0x3: INT_CARRY(register_space + 0x200ul /* CF */, 1ul, register_space + 0x0ul /* EAX */, 4ul, (uint8_t *)&(long unsigned int){0x1ul}

We can compile using gcc, or run through cbmc or AFL or what have you.

It requires some harness setup to do anything with it though. We need to allocate memory for ram, registers, and temporary values. This is handled in the `init_CPUState` library function.

We set `state.pc` to `0x1169` to go to the entrypoint of the `all_equal`. We can also set some registers. The `x86.h` header file is autogenerated and has define pcode register offsets for the architecture.

`pcode2c(&state,-1);` runs the interpreter without a breakpoint set. It is currently setup to return at every CALL or RETURN pcode instruction. In this case, it returns at the end of the function.

In [ ]:
%%file /tmp/harness.c
#include "all_equal_pcode.c"
#include "x86.h"
#include <stdlib.h>

int main() {
  CPUState state;
  init_CPUState(&state);

  // Initialize state here, including pc address.
  state.pc = 0x1169;

  memset(state.reg + RDI, 1, 8); // n 
  memset(state.reg + RSI, 64, 8); // a
  memset(state.reg + RDX, 64, 8); // b
  memset(state.reg + RCX, 0, 8);
  memset(state.reg + RAX, 0, 8);
  memset(state.reg + RSP, 0, 8);

  pcode2c(&state, -1); // Run decompiled function
  printf("RAX: %ld\n", *(uint64_t *)(state.reg + RAX));
  __CPROVER_printf("RAX: %ld\n", *(uint64_t *)(state.reg + RAX));
  assert(*(uint8_t *)(state.reg + RAX) != 0);

  return 0;
}


In [16]:
! cbmc --unwinding-assertions --bounds-check --pointer-check /tmp/all_equal.c

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/all_equal.c
Converting
Type-checking all_equal
file /tmp/all_equal.c line 13 function main: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Unwinding loop equal.0 iteration 1 file /tmp/all_equal.c line 3 function equal thread 0
Unwinding loop equal.0 iteration 2 file /tmp/all_equal.c line 3 function equal thread 0
Unwinding loop equal.0 iteration 3 file /tmp/all_equal.c line 3 function equal thread 0
Unwinding loop equal.0 iteration 4 file /tmp/all_equal.c line 3 function equal thread 0
Unwinding loop equal.0 iteration 5 file /tmp/all_equal.c line 3 function equal thread 0
Unwinding loop equal.0 iteration 6 file /tmp/all_equal.c line 3 function equal thread 0
Unwinding loop equal.0 iteration 7 file /tmp/all_equal

Because I exported debug data with `-g` when I called `gcc`, there is useful translation validation data in the debug section.  Let's take a look at the big dump. Scroll on past and I'll point out some stuff

In [17]:
!objdump -W /tmp/all_equal


/tmp/all_equal:     file format elf64-x86-64

Contents of the .eh_frame section:


00000000 0000000000000014 00000000 CIE
  Version:               1
  Augmentation:          "zR"
  Code alignment factor: 1
  Data alignment factor: -8
  Return address column: 16
  Augmentation data:     1b
  DW_CFA_def_cfa: r7 (rsp) ofs 8
  DW_CFA_offset: r16 (rip) at cfa-8
  DW_CFA_nop
  DW_CFA_nop

00000018 0000000000000014 0000001c FDE cie=00000000 pc=0000000000001080..00000000000010a6
  DW_CFA_advance_loc: 4 to 0000000000001084
  DW_CFA_undefined: r16 (rip)
  DW_CFA_nop
  DW_CFA_nop
  DW_CFA_nop
  DW_CFA_nop

00000030 0000000000000024 00000034 FDE cie=00000000 pc=0000000000001020..0000000000001050
  DW_CFA_def_cfa_offset: 16
  DW_CFA_advance_loc: 6 to 0000000000001026
  DW_CFA_def_cfa_offset: 24
  DW_CFA_advance_loc: 10 to 0000000000001030
  DW_CFA_def_cfa_expression (DW_OP_breg7 (rsp): 8; DW_OP_breg16 (rip): 0; DW_OP_lit15; DW_OP_and; DW_OP_lit10; DW_OP_ge; DW_OP_lit3; DW_OP_shl; DW_OP_plus)
  DW_

Let's point out some interesting stuff in particular

```
 <2><16d>: Abbrev Number: 4 (DW_TAG_formal_parameter)
    <16e>   DW_AT_name        : n
    <170>   DW_AT_decl_file   : 1
    <170>   DW_AT_decl_line   : 3
    <170>   DW_AT_decl_column : 16
    <171>   DW_AT_type        : <0x118>
    <175>   DW_AT_location    : 1 byte block: 55        (DW_OP_reg5 (rdi))
 <2><177>: Abbrev Number: 4 (DW_TAG_formal_parameter)
    <178>   DW_AT_name        : a
    <17a>   DW_AT_decl_file   : 1
    <17a>   DW_AT_decl_line   : 3
    <17a>   DW_AT_decl_column : 23
    <17b>   DW_AT_type        : <0x1ab>
    <17f>   DW_AT_location    : 1 byte block: 54        (DW_OP_reg4 (rsi))
 <2><181>: Abbrev Number: 4 (DW_TAG_formal_parameter)
    <182>   DW_AT_name        : b
    <184>   DW_AT_decl_file   : 1
    <184>   DW_AT_decl_line   : 3
    <184>   DW_AT_decl_column : 32
    <185>   DW_AT_type        : <0x1ab>
    <189>   DW_AT_location    : 1 byte block: 51        (DW_OP_reg1 (rdx))
```
These entries say that the parameters to the function are in rdi, rsi rdx, but we already expected that from the calling conventions.

More interesting is `i`

```
 <3><190>: Abbrev Number: 19 (DW_TAG_variable)
    <191>   DW_AT_name        : i
    <193>   DW_AT_decl_file   : 1
    <194>   DW_AT_decl_line   : 4
    <195>   DW_AT_decl_column : 13
    <196>   DW_AT_type        : <0x118>
    <19a>   DW_AT_location    : 0x14 (location list)
    <19e>   DW_AT_GNU_locviews: 0xc
```

In the .debug_loclists section we can see 
```
    Offset   Begin            End              Expression
    00000014 v000000000000002 v000000000000000 views at 0000000c for:
             000000000000116d 0000000000001174 (DW_OP_lit0; DW_OP_stack_value)
    0000001a v000000000000000 v000000000000000 views at 0000000e for:
             0000000000001174 000000000000118d (DW_OP_reg0 (rax))
    0000001f v000000000000000 v000000000000000 views at 00000010 for:
             000000000000118d 000000000000118e (DW_OP_reg2 (rcx))
    00000024 v000000000000000 v000000000000000 views at 00000012 for:
             000000000000118e 0000000000001193 (DW_OP_reg0 (rax))
```
Indeed looking at the assembly, these chunks correspond to pieces of the `for` loop, incrementing `i` by 1. We also use i to index into the arrays and it is temporarily see that it is copied over to `rcx` at addresses `0x117e` to `0x1182`

I don't know what it is getting at with the DW_OP_stack_value code. `i` is never on the stack in any obvious sense. Perhaps I need to dig into the DWARF spec of perhaps this is nonsense.

```
/tmp/all_equal.c:4 (discriminator 2)
    1174:	83 c0 01             	add    $0x1,%eax
/tmp/all_equal.c:4 (discriminator 1)
    1177:	39 f8                	cmp    %edi,%eax
    1179:	7d 13                	jge    118e <equal+0x25> (File Offset: 0x118e)
```



The `addr2line` command line program can map addresses to lines using this dwarf information. The table itself is bit tough to read. It's a compact representation.
I already put some flags on the `objdump` above.


In [21]:
!addr2line -a -e /tmp/all_equal 0x1169 0x1174 0x1186 0x1188

0x0000000000001169
/tmp/all_equal.c:3
0x0000000000001174
/tmp/all_equal.c:4 (discriminator 2)
0x0000000000001186
/tmp/all_equal.c:5
0x0000000000001188
/tmp/all_equal.c:6


Ok, so how can we use this to perform translation validation?

Well, I can annotate my original function with calls to the interpreter. By setting breakpoint addresses and then asserting that variables are in the appropriate level locations.

One typically annoying thing is that DWARf annotates with respect to syntactic positions. The line table can encode line and column information in the source, which is nice because it is highly generic with respect to language, but obviously a bummer because it can be very unclear semantically what a particular syntactic position corresponds to.

It is kind of cool though that also if I want to annotate the original source, it is in principle a very easy syntactic insertion. I've been writing a tool to do this but have lost steam on doing so, so I think it is best to just get the idea out the showing how this can be done manually.

Basically, I'd want my [`pcode2c.dwarf_annot`](https://github.com/philzook58/pcode2c/blob/main/pcode2c/dwarf_annot.py) tool to do something like this:

In [ ]:
%%file /tmp/all_equal_debug.c
#include <assert.h>
#include <stdbool.h>
bool equal(int n, int a[], int b[]) {
  for (int i = 0; i < n; i++) {

    pcode2c(&state, 0x117b);
    assert(i == state.reg[RAX]);
    pcode2c(&state, 0x117e); //continue
    
    if (a[i] != b[i]) {
      return false;
    }
  }
  return true;
}

While I could write `dwarf_annot` to get this started, I would anticipate it being loosy goosy enough toi need hand tweaking.

The basic idea though is this:
- For every line table entry, insert a call to `pcode2c(&state , addr );` at the syntactic position stated by the line table
- At these possitions, if we fall in the loclist range of a dwarf annotated `DW_TAG_variable`, output an assertion that the C variable has the same contents as the DWARF expression describing where it should be (`DW_OP_reg0 (rax)` for example)



# Bits and Bobbles
This post is almost more aspirational than true. I think the basic vision makes sense but I have a hard time getting cbmc to run on these examples.

I'm running out of motivation on pcode2c. There is a huge hill of getting the semantics tighter, improving usability (reducing harness code needed), increasing scale. A constant fear is CBMC not scaling. It does ok on small stuff.

The big "killer" application of this was supposed to be micropatching. You need to verify the locations of variables internal to a function if you want to hijack them with a trustworthy tiny pacth.

It is pretty nice having a hackable little interpreter you can toss print statements and whatnot into.

CBMC got some more printf support which was crucial for me to interpret their counterexample traces. I grep out the printf statements from the rest

Maybe make a standalone pcode interpreter? Ghidra has an interpreter and maybe I should work on pulling it out. It's a C++ thing. I feel like mine is more direct, but so what.

It is a confusing thing, because debug info is not guaranteed. And yet if it is possible to generate it, the compiler should.



It'd be cool to do ghidra plays mario usjing pcode2c https://github.com/nevesnunes/ghidra-plays-mario

There could be a more self contained "asmbmc". I didn't do this for two reasons
- I wanted to be decoupled from specifically cbmc's IR. I haven't really moved beyond cbmc though. It is not the obvious top  contender in the SVComp
- I wanted the flexibility to control the interpreter from C and run it through gcc
- I wanted to compare against C implementations of the same function as "spec"


# PCode2C pt 2: DWARF Verification via Ghidra and CBMC

Comparative verification is a really useful paradigm

- Spec writing. It is easier for users to grok that you are compring two programs rather than bringing in a complicated logic.
- Comper correctness

My basic model of what a dwarf correctness property is is that it should be specifying optionally observable effects.

`{impl_defined} -> State -> Option State`
`{impl_defined} -> Interaction`
`type Interaction = Out Value interaction | Input (Value -> interaction)`

`type compiler = prog1 : HighCode -> (prog2 : LowCode, exec_high prog1 ~ exec_low prog2)`

What is `~` though?
Effects
